In [ ]:
import numpy as np
import torch
import pycbc
from scipy import signal
import os
import gzip
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import gc
gc.collect()

In [ ]:
specs = []
df = pd.read_csv('data/training_labels.csv').set_index('id')

PATH = 'data/train'
files = os.listdir(PATH)
for file in files:
    with gzip.open(os.path.join(PATH,file), 'rb') as handle: 
        data = pickle.load(handle)

    window = torch.FloatTensor(signal.windows.tukey(4096+2*2048,0.5))
    spec = []
    for k in tqdm(data.keys()):
        if df.loc[k,'target']: continue
        c = data[k]
        c = torch.cat([-c.flip(-1)[:,4096-2049:-1]+2*c[:,0].unsqueeze(-1),c,
                                   -c.flip(-1)[:,1:2049]+2*c[:,-1].unsqueeze(-1)],1)
        spec.append(torch.fft.fft((1e20*c*window).cuda()).abs().cpu())
    specs.append(torch.stack(spec,0).mean(0))
s = torch.stack(specs,0)
torch.save(s.mean(0),'data/avr_w0.pth')